# Stop exits - buy limit order

### Part 1 - Dummy test

Below is the simple illustration of stop loss with dummy price series datasets, so do its entries and exits signals.

- Step 1: Create the dummy datasets

In [ ]:
import pandas
from datetime import datetime
import vectorbt as vbt
from vectorbt.signals.enums import StopType

# create for entries and exits
mask = pandas.DataFrame([   
    [True, False, False],
    [False, True, False],
    [False, False, True],
    [True, False, False],
    [False, True, False],
    [False, False, False]
], index=pandas.Index([
    datetime(2020, 1, 1),
    datetime(2020, 1, 2),
    datetime(2020, 1, 3),
    datetime(2020, 1, 4),
    datetime(2020, 1, 5),
    datetime(2020, 1, 6),
]), columns=['a', 'b', 'c'])


price = pandas.DataFrame({
    'open': [10, 11, 12, 11, 10, 10],
    'high': [11, 12, 13, 12, 11, 13],
    'low': [9, 10, 11, 10, 9, 10],
    'close': [10, 11, 12, 11, 10, 11]
})
out_dict_entries={}
out_dict_exits={}

- Step 2: Use generate_ohlc_stop_exits to create buy limit

In [ ]:
# buy limit order => buy at next close price if next close price is 4% lower than current close price
entries = mask.vbt.signals.generate_ohlc_stop_exits(
    price["open"], price["high"], price["low"], price["close"],
    sl_stop=0.04, sl_trail=False,
    out_dict=out_dict_entries, exit_wait=1)

# sell limit order => exit trade at next close price if next close price is 8% higher than current close price
exits = mask.vbt.signals.generate_ohlc_stop_exits(
    price['open'], price['high'], price['low'], price['close'],
    tp_stop=0.08, 
    out_dict=out_dict_exits, exit_wait=1)

entries, exits = entries.vbt.signals.clean(exits) # remove ignored exit signals

- Step 3: Check entry & exit points

In [ ]:
entries

In [ ]:
# check buy limit price
## Note: When stop loss signals (in arrays of Boolean values, i.e., [True, False, False, True...]) 
## are introduced at ENTRY POINTS, the mechanism functions as a BUY LIMIT ORDER. 
## However, implementing these stop loss signals (in Boolean) at exit points will 
## result in the transformation of the mechanism into a stop loss order.
out_dict_entries["stop_type"].vbt(mapping=StopType).apply_mapping() +\
    " at " + out_dict_entries["stop_price"].round(2).astype(str)

- Step 3: Check the trades records

In [ ]:
pf = vbt.Portfolio.from_signals(price["close"],
                                entries,
                                exits,
                                open=price["open"],
                                high=price["high"],
                                low=price["low"],
                                init_cash=1e4,
                                direction="longonly",
                                fees=0.01, #in %
                                )
pf.trades.records_readable

In [ ]:
pf.orders.records_readable

In [ ]:
pf["a"].stats()

## Real case

In [ ]:
import numpy
import vectorbt as vbt

ohlcv = vbt.YFData.download(
    "BTC-USD",
    start='2017-01-01 UTC',
    end='2020-01-01 UTC'
).concat()

# Random enter signal generator based on the number of signals.
rand = vbt.RAND.run(ohlcv["Close"].shape, n=10, seed=42)
# Random exit signal generator based on the number of signals.
randx = vbt.RANDX.run(rand.entries, seed=42)

stops = [0.1,]
sl_exits = vbt.OHLCSTX.run(
    rand.entries, 
    ohlcv['Open'], 
    ohlcv['High'], 
    ohlcv['Low'], 
    ohlcv['Close'], 
    sl_stop=list(stops),
    stop_type=None, 
    stop_price=None
).exits
exits = randx.exits.vbt | sl_exits

pf1 = vbt.Portfolio.from_signals(ohlcv['Close'], rand.entries, exits)  # with SL
pf1.stats()

In [ ]:
import vectorbt as vbt

ohlcv = vbt.YFData.download(
    "BTC-USD",
    start='2017-01-01 UTC',
    end='2020-01-01 UTC'
).concat()

# Random enter signal generator based on the number of signals.
rand = vbt.RAND.run(ohlcv["Close"].shape, n=10, seed=42)
# Random exit signal generator based on the number of signals.
randx = vbt.RANDX.run(rand.entries, seed=42)

stop_exits = rand.entries.vbt.signals.generate_stop_exits(
    ohlcv['Close'], 
    stop=0.1, 
    trailing=False,
)
exits = randx.exits.vbt | stop_exits
entries, exits = rand.entries.vbt.signals.clean(exits)

pf4 = vbt.Portfolio.from_signals(ohlcv['Close'], entries, exits,
                                 open=ohlcv["Open"],high=ohlcv["High"],
                                 low=ohlcv["Low"])
pf4.stats()

In [ ]:
import vectorbt as vbt

ohlcv = vbt.YFData.download(
    "BTC-USD",
    start='2017-01-01 UTC',
    end='2020-01-01 UTC'
).concat()

# Random enter signal generator based on the number of signals.
rand = vbt.RAND.run(ohlcv["Close"].shape, n=10, seed=42)
# Random exit signal generator based on the number of signals.
randx = vbt.RANDX.run(rand.entries, seed=42)

stop_exits = rand.entries.vbt.signals.generate_ohlc_stop_exits(
    open=ohlcv["Open"], 
    high=ohlcv['High'],
    low=ohlcv['Low'], 
    close=ohlcv['Close'], 
    sl_stop=0.1,
    sl_trail=False,
)
exits = randx.exits.vbt | stop_exits # optional: combine exit signals such that the first exit of two conditions wins
entries, exits = rand.entries.vbt.signals.clean(exits) # optional: automatically remove ignored exit signals

pf2 = vbt.Portfolio.from_signals(ohlcv['Close'], entries, exits,
                                 open=ohlcv["Open"],high=ohlcv["High"],
                                 low=ohlcv["Low"])
pf2.stats()

In [ ]:
pf2.orders.records_readable

In [3]:
import numpy
import vectorbt as vbt

ohlcv = vbt.YFData.download(
    "BTC-USD",
    start='2017-01-01 UTC',
    end='2020-01-01 UTC'
).concat()

# Random enter signal generator based on the number of signals.
rand = vbt.RAND.run(ohlcv["Close"].shape, n=10, seed=42)
# Random exit signal generator based on the number of signals.
randx = vbt.RANDX.run(rand.entries, seed=42)

stops = [0.1,]
sl_exits = vbt.OHLCSTX.run(
    rand.entries, 
    ohlcv['Open'], 
    ohlcv['High'], 
    ohlcv['Low'], 
    ohlcv['Close'], 
    sl_stop=list(stops),
    stop_type=None, 
    stop_price=None
).exits
exits = randx.exits.vbt | sl_exits

pf3 = vbt.Portfolio.from_signals(ohlcv['Close'], rand.entries, exits)  # with SL
pf3.stats()

Start                         2017-01-01 00:00:00+00:00
End                           2019-12-31 00:00:00+00:00
Period                               1095 days 00:00:00
Start Value                                       100.0
End Value                                    459.051539
Total Return [%]                             359.051539
Benchmark Return [%]                         620.566853
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              50.865378
Max Drawdown Duration                 510 days 00:00:00
Total Trades                                         10
Total Closed Trades                                  10
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                       60.0
Best Trade [%]                                93.069372
Worst Trade [%]                              -16

In [ ]:
# Reference: stop exits with RANDENEX indicator: https://github.com/polakowo/vectorbt/issues/181
import vectorbt as vbt

ohlcv = vbt.YFData.download(
    "BTC-USD",
    start='2017-01-01 UTC',
    end='2020-01-01 UTC'
).concat()
# Random enter signal generator based on the number of signals.
rand = vbt.RAND.run(ohlcv["Close"].shape, n=10, seed=42)
# Random exit signal generator based on the number of signals.
randx = vbt.RANDX.run(rand.entries, seed=42)
pf3 = vbt.Portfolio.from_signals(ohlcv["Close"], 
                                rand.entries, 
                                randx.exits,
                                open=ohlcv["Open"],
                                high=ohlcv["High"],
                                low=ohlcv["Low"],
                                sl_stop=0.1,
                                sl_trail=False,
                                )
pf3.stats()

In [ ]:
pf3.orders.records_readable